In [2]:
import pandas as pd
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [4]:
data = pd.read_excel(r"C:\Users\a\Desktop\mini_project\14_악성댓글\toxic.xlsx")
train, test = train_test_split(data, test_size=0.2, random_state=777)

In [6]:
model_name = "klue/bert-base"  
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

c:\Users\a\anaconda3\envs\NLP_38_018\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\a\.cache\huggingface\hub\models--klue--bert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# 데이터셋 구축 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, tokenizer, max_len):
        self.dataset = dataset
        self.sent_idx = sent_idx
        self.label_idx = label_idx
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, i):
        sentence = str(self.dataset.iloc[i, self.sent_idx])
        label = self.dataset.iloc[i, self.label_idx]

        inputs = self.tokenizer(
            sentence,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )
        input_ids = inputs["input_ids"].squeeze()
        attention_mask = inputs["attention_mask"].squeeze()
        return input_ids, attention_mask, torch.tensor(label)

    def __len__(self):
        return len(self.dataset)

In [9]:
max_len = 100
batch_size = 32
learning_rate = 5e-5
num_epochs = 3

In [10]:
dataset_train = BERTDataset(train, 0, 1, tokenizer, max_len)
dataset_test = BERTDataset(test, 0, 1, tokenizer, max_len)

In [11]:
train_dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

In [12]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
loss_fn = nn.CrossEntropyLoss()

c:\Users\a\anaconda3\envs\NLP_38_018\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [15]:
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_dataloader):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    # 검증 루프 (선택)
    model.eval()
    eval_accuracy = 0
    eval_steps = 0
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        eval_accuracy += (preds == labels).sum().item()
        eval_steps += len(labels)

    print(f"Epoch {epoch + 1}, Accuracy: {eval_accuracy / eval_steps}")

  0%|          | 0/250 [00:00<?, ?it/s]

Epoch 1, Accuracy: 0.7975


  0%|          | 0/250 [00:00<?, ?it/s]

KeyboardInterrupt: 